In [2]:
import silq
silq.initialize("EWJN", mode='analysis')

Initializing imports
Initializing functions
Initializing data
Initializing analysis
Initializing plotting
Initialization complete


In [3]:
from silq.measurements.measurement_modules import *
from silq.measurements.measurement_types import *

In [4]:
adiabatic_ESR_parameter = ManualParameter(name='adiabatic_ESR', initial_value=42)
dummy_parameter = ManualParameter(name='dummy', initial_value=42)
dummy_parameter2 = ManualParameter(name='dummy2', initial_value=42)
adiabatic_ESR_parameter.get = np.random.random

condition_sets = [ConditionSet(('contrast', '>', 0.4), on_success='update')]
measurements = [Loop2DMeasurement('detailed', acquisition_parameter=adiabatic_ESR_parameter),
                Loop2DMeasurement('rough', acquisition_parameter=adiabatic_ESR_parameter)]
measurement_sequence = MeasurementSequence('retune', measurements=measurements,
                                           condition_sets=condition_sets)

# Loop measurements

## Loop 0D measurement

In [5]:
loop0D_msmt = Loop0DMeasurement('loop_0D', acquisition_parameter=adiabatic_ESR_parameter, 
                                base_folder='2016-12-29/#028_test_measure_adiabatic_ESR_16-28-37')
# data =loop0D_msmt()

## Loop 1D measurement

In [22]:
loop1D_msmt = Loop1DMeasurement('loop_1D', acquisition_parameter=adiabatic_ESR_parameter, 
                                base_folder='2016-12-29/#028_test_measure_adiabatic_ESR_16-28-37',
                                set_parameter=dummy_parameter)
loop1D_msmt(range(10))
# data =loop1D_msmt()

data = loop1D_msmt(step=1, points=10)
data =loop1D_msmt()

DataSet:
   mode     = DataMode.LOCAL
   location = '2016-12-29/#028_test_measure_adiabatic_ESR_16-28-37/#084_loop_1D_dummy_adiabatic_ESR_18-09-55'
   <Type>   | <array_id>    | <array.name>  | <array.shape>
   Setpoint | dummy_set     | dummy         | (10,)
   Measured | adiabatic_ESR | adiabatic_ESR | (10,)
started at 2016-12-29 18:09:55


In [23]:
condition_set = ConditionSet(('adiabatic_ESR', '>', 0.9))
loop1D_msmt.check_condition_sets(condition_set)

{'action': None,
 'is_satisfied': True,
 'measurement': 'loop_1D',
 'satisfied_arr': array([False, False, False, False, False, False, False,  True, False, False], dtype=bool)}

In [24]:
data.adiabatic_ESR

DataArray[10]: adiabatic_ESR
array([ 0.45 ,  0.744,  0.297,  0.049,  0.191,  0.605,  0.181,  0.935,
        0.362,  0.227])

## Loop 2D measurement

In [9]:
loop2D_msmt = Loop2DMeasurement('loop_1D', acquisition_parameter=adiabatic_ESR_parameter, 
                                base_folder='2016-12-29/#028_test_measure_adiabatic_ESR_16-28-37',
                                set_parameters=[dummy_parameter, dummy_parameter2])
loop2D_msmt(steps=1, points=10)
data =loop2D_msmt()

DataSet:
   mode = DataMode.LOCAL
   data = qc.load_data('2016-12-29/#028_test_measure_adiabatic_ESR_16-28-37/#085_loop_1D_dummy_dummy2_adiabatic_ESR_09-30-13')
   <Type>   | <array_id>    | <array.name>  | <array.shape>
   Setpoint | dummy_set     | dummy         | (10,)
   Setpoint | dummy2_set    | dummy2        | (10, 10)
   Measured | adiabatic_ESR | adiabatic_ESR | (10, 10)
started at 2016-12-31 09:30:14
